In [1]:
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import process

/Users/hedvikapajerova/venv/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Načtení datasetu s podrobnostmi o knihách. Různá vydání nás v tuto chvíli nezajímají, proto odstraním duplicitní názvy.

In [2]:
books_df = pd.read_csv("BX-Books.csv", encoding="latin-1", on_bad_lines="skip", sep=';')
books_df = books_df.drop_duplicates(subset=["Book-Title"])
books_df

/Users/hedvikapajerova/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
271354,0449906736,Flashpoints: Promise and Peril in a New World,Robin Wright,1993,Ballantine Books,http://images.amazon.com/images/P/0449906736.0...,http://images.amazon.com/images/P/0449906736.0...,http://images.amazon.com/images/P/0449906736.0...
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...


Načtení datasetu s podrobnostmi o uživatelích.

In [3]:
users_df = pd.read_csv("BX-Users.csv", encoding="latin-1", on_bad_lines="skip", sep=';')
users_df

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN
...,...,...,...
278853,278854,"portland, oregon, usa",NaN
278854,278855,"tacoma, washington, united kingdom",50.0
278855,278856,"brampton, ontario, canada",NaN
278856,278857,"knoxville, tennessee, usa",NaN


Načtení datasetu s hodnocením.

In [4]:
books_with_ratings_df = pd.read_csv("BX-Book-Ratings.csv", encoding="latin-1", on_bad_lines="skip", sep=';')
books_with_ratings_df

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


Budu potřebovat jen název knihy, ISBN, id uživatele a hodnocení knihy. Sloučím tedy dataset s údaji o knihách a dataset s hodnocením podle sloupce ISBN a zbytečné sloupce vymažu.

In [5]:
books_ratings_titles_complete_df = books_df.join(books_with_ratings_df.set_index("ISBN"), on="ISBN")
books_ratings_titles_complete_df = books_ratings_titles_complete_df.reset_index()
books_ratings_titles_complete_df = books_ratings_titles_complete_df.drop(columns=["index", "Book-Author", "Year-Of-Publication", "Publisher", "Image-URL-S", "Image-URL-M", "Image-URL-L"])
books_ratings_titles_complete_df

,ISBN,Book-Title,User-ID,Book-Rating
0,0195153448,Classical Mythology,2.0,0.0
1,0002005018,Clara Callan,8.0,5.0
2,0002005018,Clara Callan,11400.0,0.0
3,0002005018,Clara Callan,11676.0,8.0
4,0002005018,Clara Callan,41385.0,0.0
...,...,...,...,...
884148,0449906736,Flashpoints: Promise and Peril in a New World,276313.0,5.0
884149,0525447644,From One to One Hundred,276579.0,4.0
884150,006008667X,Lily Dale : The True Story of the Town that Ta...,276680.0,0.0
884151,0192126040,Republic (World's Classics),276680.0,0.0


Příprava dat pro doporučení, pokud chce uživatel doporučit četbu na základě málo hodnocené knihy, viz funkce recommender níže.

In [19]:
best_rated_books = pd.DataFrame(books_ratings_titles_complete_df.groupby("Book-Title")["Book-Rating"].median())
best_rated_books["num_of_ratings"] = pd.DataFrame(books_ratings_titles_complete_df.groupby("Book-Title")["Book-Rating"].count())
best_rated_books = best_rated_books[best_rated_books.num_of_ratings > 50]
best_rated_books = pd.DataFrame(best_rated_books.sort_values("Book-Rating",ascending=False).head(10))
best_rated_books

,Book-Rating,num_of_ratings
Book-Title,,
Harry Potter and the Prisoner of Azkaban (Book 3),9.0,197
Harry Potter and the Goblet of Fire (Book 4),9.0,194
Free,9.0,54
The Stand (The Complete and Uncut Edition),9.0,57
The Little Prince,8.0,79
Sister of My Heart,8.0,63
Griffin &amp; Sabine: An Extraordinary Correspondence,8.0,72
The Cat in the Hat,8.0,52
The Godfather,8.0,54


Pivot tabulka (viz níže) se mi nedařila z kompletního datasetu kvůli omezenému výkonu vytvořit a omezení podle počtu recenzí nefungovalo dobře s NN. Proto si tentokrát dataset omezím jiným způsobem a to tak, aby byly co nejrovnoměrněji zastoupeny recenze s jistým počtem bodů.

In [7]:
books_ratings_df_0 = books_ratings_titles_complete_df[books_ratings_titles_complete_df["Book-Rating"] == 0]
books_ratings_df_0 = books_ratings_df_0.sample(n=5052)

In [8]:
books_ratings_df_1 = books_ratings_titles_complete_df[books_ratings_titles_complete_df["Book-Rating"] == 1]
books_ratings_df_1 = books_ratings_df_1.sample(n=1302)

In [9]:
books_ratings_df_2 = books_ratings_titles_complete_df[books_ratings_titles_complete_df["Book-Rating"] == 2]
books_ratings_df_2 = books_ratings_df_2.sample(n=2037)

In [10]:
books_ratings_df_3 = books_ratings_titles_complete_df[books_ratings_titles_complete_df["Book-Rating"] == 3]
books_ratings_df_3 = books_ratings_df_3.sample(n=4376)

In [11]:
books_ratings_df_4 = books_ratings_titles_complete_df[books_ratings_titles_complete_df["Book-Rating"] == 4]
books_ratings_df_4 = books_ratings_df_4.sample(n=5052)

In [12]:
books_ratings_df_5 = books_ratings_titles_complete_df[books_ratings_titles_complete_df["Book-Rating"] == 5]
books_ratings_df_5 = books_ratings_df_5.sample(n=5052)

In [13]:
books_ratings_df_6 = books_ratings_titles_complete_df[books_ratings_titles_complete_df["Book-Rating"] == 6]
books_ratings_df_6 = books_ratings_df_6.sample(n=5052)

In [14]:
books_ratings_df_7 = books_ratings_titles_complete_df[books_ratings_titles_complete_df["Book-Rating"] == 7]
books_ratings_df_7 = books_ratings_df_7.sample(n=5052)

In [15]:
books_ratings_df_8 = books_ratings_titles_complete_df[books_ratings_titles_complete_df["Book-Rating"] == 8]
books_ratings_df_8 = books_ratings_df_8.sample(n=5052)

In [16]:
books_ratings_df_9 = books_ratings_titles_complete_df[books_ratings_titles_complete_df["Book-Rating"] == 9]
books_ratings_df_9 = books_ratings_df_9.sample(n=5052)

In [17]:
books_ratings_df_10 = books_ratings_titles_complete_df[books_ratings_titles_complete_df["Book-Rating"] == 10]
books_ratings_df_10 = books_ratings_df_10.sample(n=5052)

In [20]:
frames = [books_ratings_df_0, books_ratings_df_1, books_ratings_df_2, books_ratings_df_3, books_ratings_df_4, books_ratings_df_5, books_ratings_df_6, books_ratings_df_7, books_ratings_df_8, books_ratings_df_9, books_ratings_df_10]
books_ratings_df = pd.concat(frames)
books_ratings_df = books_ratings_df.sort_values(by=["Book-Rating"])
books_ratings_df["Book-Rating"] = books_ratings_df["Book-Rating"].astype(int)
books_ratings_df

,ISBN,Book-Title,User-ID,Book-Rating
98302,0449003787,"\O\"" Is for Outlaw""",23768.0,0
64026,0743225406,The Shipping News : A Novel,38437.0,0
81014,0140004424,Brighton Rock,254899.0,0
822258,0844270741,Una mirada a Espana,82831.0,0
493956,0440234719,The Texan,27462.0,0
...,...,...,...,...
314650,076422249X,"The Sacred Shore (Oke, Janette, Song of Acadia...",200188.0,10
315449,0679732187,"Absalom, Absalom!: The Corrected Text (Vintage...",129716.0,10
78186,0385512104,The Curious Incident of the Dog in the Night-T...,235105.0,10
225402,0312991460,To the Nines (A Stephanie Plum Novel),62209.0,10


Pivot tabulka: Podklad pro výpočet csr matice.

In [21]:
books_ratings_df_pivot = books_ratings_df.pivot_table(index="Book-Title", columns="User-ID", values="Book-Rating").fillna(0)
books_ratings_df_pivot

User-ID,8.0,17.0,22.0,53.0,81.0,82.0,85.0,99.0,114.0,141.0,...,278648.0,278698.0,278732.0,278759.0,278773.0,278818.0,278843.0,278844.0,278851.0,278854.0
Book-Title,,,,,,,,,,,,,,,,,,,,,
"Earth Prayers From around the World: 365 Prayers, Poems, and Invocations for Honoring the Earth",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Goosebumps Monster Edition 1: Welcome to Dead House, Stay Out of the Basement, and Say Cheese and Die!",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Q-Space (Star Trek The Next Generation, Book 47)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Q-Zone (Star Trek The Next Generation, Book 48)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Tales of Terror and Suspense,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ã? sucesso solo 50 anni fa: Lo sterminio di sei milioni di Ebrei,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ã?kotopia: Notizen und Reportagen von William Weston aus dem Jahre 1999 [sic],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ã?ngeles fugaces (Falling Angels),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Pro rychlejší výpočet převedení do csr matice.

In [23]:
mat_books_users = csr_matrix(books_ratings_df_pivot.values)

Model NN.

In [24]:
model = NearestNeighbors(metric="cosine", algorithm="brute", n_neighbors=10)

In [25]:
model.fit(mat_books_users)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=10)

Funkce na doporučení knih.

In [26]:
def recommender(book_name, number_of_books_to_recommend):
        
    if book_name in books_ratings_df.values:
        
        number_of_books_to_recommend += 1
        data = mat_books_users
        model.fit(data)

        idx = process.extractOne(book_name, books_ratings_df['Book-Title'])[2]
        print("Book selected: ", book_name)
        print("Recommended books: ")
        distances, indices=model.kneighbors(data[idx], n_neighbors=number_of_books_to_recommend)

        for i in indices:
            recommendations = books_ratings_df['Book-Title'].values[i]
            recommendations = recommendations.tolist()
                    
            for book in recommendations:
                
                if book != book_name:
                    print(book)
            
    elif book_name in books_df.values:
        print("Book selected: ", book_name)
        print("This book has not been sufficiently evaluated yet to recommend similar books.\nWhy not read some of the best rated books?")
        print(best_rated_books.drop(columns=['num_of_ratings']))
            
    else:
      print("Book not found. Please try again.")

Test: Kniha, která není v seznamu všech knih. Výsledkem je chybová hláška.

In [27]:
recommender("Antifragile", 10)

Book not found. Please try again.


Test: Kniha, která je ve finálním datasetu.

In [28]:
recommender("To Kill a Mockingbird", 10)

Book selected:  To Kill a Mockingbird
Recommended books: 
Skylock
The Sherbrooke Twins
Rebellion (StarGate, Book 1)
Anno Dracula
Forever and Always
Trusting Daddy
The Other Shepards
Miracle Man: A Novel
The Global Myths: Exploring Primitive, Pagan, Sacred, and Scientific Mythologies (Meridian S.)
Interview with the Vampire
Wild Animus


Test: Kniha, která není ve finálním datasetu. Výstupem je výběr knih hodnocených alespoň 0x s nejlepším mediánem všech hodnocení.

In [30]:
recommender("Classical Mythology", 10)

Book selected:  Classical Mythology
This book has not been sufficiently evaluated yet to recommend similar books.
Why not read some of the best rated books?
                                                    Book-Rating
Book-Title                                                     
Harry Potter and the Prisoner of Azkaban (Book 3)           9.0
Harry Potter and the Goblet of Fire (Book 4)                9.0
Free                                                        9.0
The Stand (The Complete and Uncut Edition)                  9.0
The Little Prince                                           8.0
Sister of My Heart                                          8.0
Griffin &amp; Sabine: An Extraordinary Correspo...          8.0
The Cat in the Hat                                          8.0
The Godfather                                               8.0
The Hobbit: or There and Back Again                         8.0
